In [5]:
import json
import numpy as np
from typing import Dict, List

EPS = 1e-6

def load_json(file_path: str) -> Dict:
    with open(file_path, 'r') as f:
        return json.load(f)

def extract_scores(data: Dict) -> Dict[str, List[int]]:
    return {k: [item['per_class_scores'][k] for item in data.values()]
            for k in data['0']['per_class_scores'].keys()}

def cohens_kappa(scores1: Dict[str, List[int]], scores2: Dict[str, List[int]]) -> Dict[str, float]:
    def kappa(a: np.ndarray, b: np.ndarray) -> float:
        n = len(a)
        categories = np.unique(np.concatenate([a, b]))
        n_categories = len(categories)
        
        # Observed agreement
        observed = np.sum(a == b) / n
        
        # Expected agreement
        expected = sum((np.sum(a == i) / n) * (np.sum(b == i) / n) for i in categories)
        
        # Compute kappa
        kappa = (observed - expected) / (1 - expected + EPS)
        return kappa

    results = {}
    for category in scores1.keys():
        a = np.array(scores1[category])
        b = np.array(scores2[category])
        results[category] = kappa(a, b)
    
    return results

def compute_kappa_for_files(file1: str, file2: str) -> Dict[str, float]:
    data1 = load_json(file1)
    data2 = load_json(file2)

    # Find overlapping keys
    overlap = set(data1.keys()) & set(data2.keys())
    data1_overlap, data2_overlap = {}, {}
    for key in overlap:
        data1_overlap[key] = data1[key]
        data2_overlap[key] = data2[key]
    scores1 = extract_scores(data1_overlap)
    scores2 = extract_scores(data2_overlap)
    
    return cohens_kappa(scores1, scores2)

# Example usage
file1 = 'manual_labels_can.json'
file2 = 'manual_labels_adam.json'
kappa_scores = compute_kappa_for_files(file1, file2)

print("Cohen's Kappa scores for each category:")
for category, score in kappa_scores.items():
    print(f"{category}: {score:.4f}")

Cohen's Kappa scores for each category:
male / female: 0.5000
professor / nurse: 0.2593
male_professor / female_nurse: 0.2958
accountant: -0.1538
architect: 0.0000
attorney: 0.0000
dentist: 0.0000
filmmaker: 0.0000
